In [ ]:
import string
import re
import os
import nltk
import pandas as pd
import numpy as np
import json
import glob
import gzip
import tensorflow as tf
from tensorflow import keras
import time

import matplotlib.pyplot as plt

SEED = 1013
np.random.seed(SEED)

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from tensorflow.keras import Model
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer


import pandas as pd 
import numpy as np 
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from scipy import sparse
import os
import pickle
import emoji
from wordcloud import WordCloud

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
from nltk.util import ngrams

nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer

In [ ]:
import collections
df=pd.read_csv('survey_data')
responses=df.iloc[2:,12:]

reordered_responses=responses[['Q34','Q58','Q12','Q13','Q14','Q16','Q35','Q18','Q19','Q20','Q21','Q30','Q34','Q17','Q15','Q1','Q39','Q28','Q3','Q4','Q5','Q6','Q9','Q29_1','Q29_2','Q29_3','Q29_4','Q29_5','Q29_6','Q29_7']]

free_form_responses=reordered_responses.iloc[:,:12]

single_choice_responses=reordered_responses.iloc[:,12:]

individual_responses=collections.defaultdict(list)
free_form_responses_without_race=free_form_responses.iloc[:,1:]
race=free_form_responses.iloc[:,0].tolist()
r=-1
for index, row in free_form_responses_without_race.iterrows():
    r+=1
    for i in range(len(row)):
        try:
            if float(row[i])==row[i]:
                continue
        except ValueError:
            individual_responses[(index,race[r])].append(row[i])

In [ ]:
print(len(individual_responses))

In [ ]:
survey_data_raw = []
survey_labels = []
for i in individual_responses:
    for z in individual_responses[i]:
        survey_data_raw.append(z)
        if i[1] == 'Black or African American':
            survey_labels.append(1)
        else:
            survey_labels.append(0)

In [ ]:
survey_box_plot = [len(t) for t in survey_data_raw]

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
plt.boxplot(survey_box_plot)
plt.show()

In [ ]:
print(survey_data_raw[4])

In [ ]:
print(len(survey_data_raw))
print(len(set(survey_data_raw)))
print(len(survey_labels))

In [ ]:
pd.set_option('display.max_colwidth', None)
df1 = pd.DataFrame(columns=['text','label'])
df1['text'] =  survey_data_raw
df1['label'] = survey_labels
df1['pre_clean_len'] = [len(t) for t in df1.text]
df1[df1.pre_clean_len > 280]

In [ ]:
plt.style.use('fivethirtyeight')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import re
from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer
tok = WordPunctTokenizer()

pat1 = r'@[A-Za-z0-9_]+'
pat2 = r'https?://[^ ]+'
combined_pat = r'|'.join((pat1, pat2))
www_pat = r'www.[^ ]+'
negations_dic = {"isn't":"is not", "aren't":"are not", "wasn't":"was not", "weren't":"were not",
                "haven't":"have not","hasn't":"has not","hadn't":"had not","won't":"will not",
                "wouldn't":"would not", "don't":"do not", "doesn't":"does not","didn't":"did not",
                "can't":"can not","couldn't":"could not","shouldn't":"should not","mightn't":"might not",
                "mustn't":"must not"}
neg_pattern = re.compile(r'\b(' + '|'.join(negations_dic.keys()) + r')\b')

def tweet_cleaner_updated(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    try:
        bom_removed = souped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        bom_removed = souped
    stripped = re.sub(combined_pat, '', bom_removed)
    stripped = re.sub(www_pat, '', stripped)
    stripped = re.sub(r'\@w+','',stripped)
    lower_case = stripped.lower()
    neg_handled = neg_pattern.sub(lambda x: negations_dic[x.group()], lower_case)
    letters_only = re.sub("[^a-zA-Z]", " ", neg_handled)
    words = [x for x  in tok.tokenize(letters_only) if len(x) > 1]
    return (" ".join(words)).strip()

In [ ]:
train_data = []
for data in survey_data_raw:
    train_data.append(tweet_cleaner_updated(data))
    
train_labels = survey_labels

In [ ]:
survey_box_plot2 = [len(t) for t in train_data]
fig, ax = plt.subplots(figsize=(5, 5))
plt.boxplot(survey_box_plot2)
plt.show()

In [ ]:
df = pd.DataFrame(columns=['text','label'])

df['text'] = train_data
df['label'] = train_labels

df0 = df[df["label"] == 0] 
df1 = df[df["label"] == 1]


In [ ]:
neg_tweets = df[df.label == 0]
neg_string = []
for t in neg_tweets.text:
    neg_string.append(t)
neg_string = pd.Series(neg_string).str.cat(sep=' ')

wordcloud = WordCloud(width=1600, height=800,max_font_size=200).generate(neg_string)
plt.figure(figsize=(12,10))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
pos_tweets = df[df.label == 1]
pos_string = []
for t in pos_tweets.text:
    pos_string.append(t)
pos_string = pd.Series(pos_string).str.cat(sep=' ')
wordcloud = WordCloud(width=1600, height=800,max_font_size=200,colormap='magma').generate(pos_string) 
plt.figure(figsize=(12,10)) 
plt.imshow(wordcloud, interpolation="bilinear") 
plt.axis("off") 
plt.show()

In [ ]:
cvec = CountVectorizer()
cvec.fit(df.text)

In [ ]:
len(cvec.get_feature_names())

In [ ]:
neg_doc_matrix = cvec.transform(df[df.label == 0].text)
pos_doc_matrix = cvec.transform(df[df.label == 1].text)
neg_tf = np.sum(neg_doc_matrix,axis=0)
pos_tf = np.sum(pos_doc_matrix,axis=0)
neg = np.squeeze(np.asarray(neg_tf))
pos = np.squeeze(np.asarray(pos_tf))
term_freq_df = pd.DataFrame([neg,pos],columns=cvec.get_feature_names()).transpose()

In [ ]:
term_freq_df.columns = ['negative', 'positive']
term_freq_df['total'] = term_freq_df['negative'] + term_freq_df['positive']
term_freq_df.sort_values(by='total', ascending=False).iloc[:10]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cvec = CountVectorizer(stop_words='english',max_features=3000)
cvec.fit(df.text)

In [ ]:
document_matrix = cvec.transform(df.text)

In [ ]:
neg_doc_matrix2 = cvec.transform(df[df.label == 0].text)
pos_doc_matrix2 = cvec.transform(df[df.label == 1].text)
neg_tf2 = np.sum(neg_doc_matrix2,axis=0)
pos_tf2 = np.sum(pos_doc_matrix2,axis=0)
neg2 = np.squeeze(np.asarray(neg_tf2))
pos2 = np.squeeze(np.asarray(pos_tf2))
term_freq_df2 = pd.DataFrame([neg2,pos2],columns=cvec.get_feature_names()).transpose()
term_freq_df2.columns = ['negative', 'positive']
term_freq_df2['total'] = term_freq_df2['negative'] + term_freq_df2['positive']
term_freq_df2.sort_values(by='total', ascending=False).iloc[:30]

In [ ]:
import seaborn as sns
plt.figure(figsize=(8,6))
ax = sns.regplot(x="negative", y="positive",fit_reg=False, scatter_kws={'alpha':0.5},data=term_freq_df2)
plt.ylabel('Positive Frequency')
plt.xlabel('Negative Frequency')
plt.title('Negative Frequency vs Positive Frequency')

In [ ]:
term_freq_df2['pos_rate'] = term_freq_df2['positive'] * 1./term_freq_df2['total']
term_freq_df2.sort_values(by='pos_rate', ascending=False).iloc[:10]

In [ ]:
term_freq_df2['neg_rate'] = term_freq_df2['negative'] * 1./term_freq_df2['total']
term_freq_df2.sort_values(by='neg_rate', ascending=False).iloc[:10]

In [ ]:
term_freq_df2['pos_freq_pct'] = term_freq_df2['positive'] * 1./term_freq_df2['positive'].sum()
term_freq_df2.sort_values(by='pos_freq_pct', ascending=False).iloc[:10]

In [ ]:
term_freq_df2['neg_freq_pct'] = term_freq_df2['negative'] * 1./term_freq_df2['negative'].sum()
term_freq_df2.sort_values(by='neg_freq_pct', ascending=False).iloc[:10]

In [ ]:
from scipy.stats import hmean

term_freq_df2['pos_hmean'] = term_freq_df2.apply(lambda x: (hmean([x['pos_rate'], x['pos_freq_pct']])
                                                                   if x['pos_rate'] > 0 and x['pos_freq_pct'] > 0 
                                                                   else 0), axis=1)                                                        
term_freq_df2.sort_values(by='pos_hmean', ascending=False).iloc[:10]

In [ ]:
term_freq_df2['neg_hmean'] = term_freq_df2.apply(lambda x: (hmean([x['neg_rate'], x['neg_freq_pct']])
                                                                   if x['neg_rate'] > 0 and x['neg_freq_pct'] > 0 
                                                                   else 0), axis=1)                                                        
term_freq_df2.sort_values(by='neg_hmean', ascending=False).iloc[:10]

In [ ]:
from scipy.stats import norm
def normcdf(x):
    return norm.cdf(x, x.mean(), x.std())
term_freq_df2['pos_rate_normcdf'] = normcdf(term_freq_df2['pos_rate'])
term_freq_df2['pos_freq_pct_normcdf'] = normcdf(term_freq_df2['pos_freq_pct'])
term_freq_df2['pos_normcdf_hmean'] = hmean([term_freq_df2['pos_rate_normcdf'], term_freq_df2['pos_freq_pct_normcdf']])
term_freq_df2.sort_values(by='pos_normcdf_hmean', ascending=False).iloc[:10]

In [ ]:
from scipy.stats import norm
def normcdf(x):
    return norm.cdf(x, x.mean(), x.std())

term_freq_df2['neg_rate_normcdf'] = normcdf(term_freq_df2['neg_rate'])
term_freq_df2['neg_freq_pct_normcdf'] = normcdf(term_freq_df2['neg_freq_pct'])
term_freq_df2['neg_normcdf_hmean'] = hmean([term_freq_df2['neg_rate_normcdf'], term_freq_df2['neg_freq_pct_normcdf']])
term_freq_df2.sort_values(by='neg_normcdf_hmean', ascending=False).iloc[:10]

In [ ]:
plt.figure(figsize=(8,6))
ax = sns.regplot(x="neg_freq_pct", y="pos_freq_pct",fit_reg=False, scatter_kws={'alpha':0.5},data=term_freq_df2)
plt.ylabel('Positive Frequency Rate')
plt.xlabel('Negative Frequency Rate')
plt.title('neg_freq_pct vs pos_freq_pct')

In [ ]:
plt.figure(figsize=(8,6))
ax = sns.regplot(x="neg_hmean", y="pos_hmean",fit_reg=False, scatter_kws={'alpha':0.5},data=term_freq_df2)
plt.ylabel('Positive Rate and Frequency Harmonic Mean')
plt.xlabel('Negative Rate and Frequency Harmonic Mean')
plt.title('neg_hmean vs pos_hmean')

In [ ]:
plt.figure(figsize=(8,6))
ax = sns.regplot(x="neg_normcdf_hmean", y="pos_normcdf_hmean",fit_reg=False, scatter_kws={'alpha':0.5},data=term_freq_df2)
plt.ylabel('Positive Rate and Frequency CDF Harmonic Mean')
plt.xlabel('Negative Rate and Frequency CDF Harmonic Mean')
plt.title('neg_normcdf_hmean vs pos_normcdf_hmean')

In [ ]:
from bokeh.plotting import figure
from bokeh.io import output_notebook, show
from bokeh.models import LinearColorMapper
output_notebook()
from bokeh.resources import INLINE
import bokeh.io


color_mapper = LinearColorMapper(palette='Inferno256', low=min(term_freq_df2.pos_normcdf_hmean), high=max(term_freq_df2.pos_normcdf_hmean))
p = figure(x_axis_label='neg_normcdf_hmean', y_axis_label='pos_normcdf_hmean')
p.circle('neg_normcdf_hmean','pos_normcdf_hmean',size=5,alpha=0.3,source=term_freq_df2,color={'field': 'pos_normcdf_hmean', 'transform': color_mapper})
from bokeh.models import HoverTool
hover = HoverTool(tooltips=[('token','@index')])
p.add_tools(hover)
show(p)

In [ ]:
train_corpus,test_corpus,train_labels,test_labels = train_test_split(train_data,train_labels,stratify=train_labels,test_size=0.25,random_state=1)
train_texts, train_labels, test_texts, test_labels = train_corpus, train_labels, test_corpus, test_labels
x_train, y_train, x_validation, y_validation = train_corpus,train_labels,test_corpus,test_labels

In [ ]:
y_train_len0 = 0
y_train_len1 = 1
for label in y_train:
    if label == 1:
        y_train_len1 += 1
    else:
        y_train_len0 += 1
        
y_val_len0 = 0
y_val_len1 = 0
for label in y_validation:
    if label == 1:
        y_val_len1 += 1
    else:
        y_val_len0 += 1

In [ ]:
print( "Train set has total {0} entries with {1:.2f}% negative, {2:.2f}% positive".format(len(x_train),
                                                                             (y_train_len0 / (len(x_train)*1.))*100,
                                                                            (y_train_len1 / (len(x_train)*1.))*100))
       
                                                                            
print("Validation set has total {0} entries with {1:.2f}% negative, {2:.2f}% positive".format(len(x_validation),
                                                                             (y_val_len0 / (len(x_validation)))*100,
                                                                            (y_val_len1 / (len(x_validation)))*100))

In [ ]:
def accuracy_summary(pipeline, x_train, y_train, x_test, y_test):
    if y_val_len0 / (len(x_test)*1.) > 0.5:
        null_accuracy =  y_val_len1/ (len(x_test)*1.)
    else:
        null_accuracy = 1. - (y_val_len1 / (len(x_test)*1.))
    #t0 = time.time()
    sentiment_fit = pipeline.fit(x_train, y_train)
    y_pred = sentiment_fit.predict(x_test)
    #train_test_time = time() - t0
    accuracy = accuracy_score(y_test, y_pred)
    print ("null accuracy: {0:.2f}%".format(null_accuracy*100))
    print ("accuracy score: {0:.2f}%".format(accuracy*100))
    if accuracy > null_accuracy:
        print ("model is {0:.2f}% more accurate than null accuracy".format((accuracy-null_accuracy)*100))
    elif accuracy == null_accuracy:
        print ("model has the same accuracy with the null accuracy")
    else:
        print ("model is {0:.2f}% less accurate than null accuracy".format((null_accuracy-accuracy)*100))
    #print ("train and test time: {0:.2f}s".format(train_test_time))
    print ("-"*80)
    return accuracy

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
cvec = CountVectorizer()
lr = LogisticRegression(max_iter=200)
n_features = np.arange(1000,4001,1000)

def nfeature_accuracy_checker(vectorizer=cvec, n_features=n_features, stop_words=None, ngram_range=(1, 1), classifier=lr):
    result = []
    print (classifier)
    print ("\n")
    for n in n_features:
        vectorizer.set_params(stop_words=stop_words, max_features=n, ngram_range=ngram_range)
        checker_pipeline = Pipeline([
            ('vectorizer', vectorizer),
            ('classifier', classifier)
        ])
        print("Validation result for {} features".format(n))
        nfeature_accuracy = accuracy_summary(checker_pipeline, x_train, y_train, x_validation, y_validation)
        result.append((n,nfeature_accuracy))
    return result

In [ ]:
from sklearn.feature_extraction import text
z = frozenset(list(term_freq_df.sort_values(by='total', ascending=False).iloc[:3].index))
c = frozenset(list(term_freq_df.sort_values(by='total', ascending=False).iloc[4:11].index))
listofsets = [z,c]
a = frozenset.union(*listofsets)
b = text.ENGLISH_STOP_WORDS
set(a).issubset(set(b))

In [ ]:
my_stop_words = a

In [ ]:
print("RESULT FOR UNIGRAM WITHOUT STOP WORDS\n")
feature_result_wosw = nfeature_accuracy_checker(stop_words='english')

In [ ]:
print("RESULT FOR UNIGRAM WITH STOP WORDS\n")
feature_result_ug = nfeature_accuracy_checker()

In [ ]:
print("RESULT FOR UNIGRAM WITHOUT CUSTOM STOP WORDS (Top 10 frequent words)\n")
feature_result_wocsw = nfeature_accuracy_checker(stop_words=my_stop_words)

In [ ]:
nfeatures_plot_ug = pd.DataFrame(feature_result_ug,columns=['nfeatures','validation_accuracy'])
nfeatures_plot_ug_wocsw = pd.DataFrame(feature_result_wocsw,columns=['nfeatures','validation_accuracy'])
nfeatures_plot_ug_wosw = pd.DataFrame(feature_result_wosw,columns=['nfeatures','validation_accuracy'])
plt.figure(figsize=(8,6))
plt.plot(nfeatures_plot_ug.nfeatures, nfeatures_plot_ug.validation_accuracy, label='with stop words')
plt.plot(nfeatures_plot_ug_wocsw.nfeatures, nfeatures_plot_ug_wocsw.validation_accuracy,label='without custom stop words')
plt.plot(nfeatures_plot_ug_wosw.nfeatures, nfeatures_plot_ug_wosw.validation_accuracy,label='without stop words')
plt.title("Without stop words VS With stop words (Unigram): Accuracy")
plt.xlabel("Number of features")
plt.ylabel("Validation set accuracy")
plt.legend()

In [ ]:
print ("RESULT FOR BIGRAM WITH STOP WORDS\n")
feature_result_bg = nfeature_accuracy_checker(stop_words=my_stop_words,ngram_range=(1, 2))

In [ ]:
print ("RESULT FOR BIGRAM WITH STOP WORDS\n")
feature_result_bg2 = nfeature_accuracy_checker(stop_words='english', ngram_range=(1, 2))

In [ ]:
print("RESULT FOR TRIGRAM WITH STOP WORDS\n")
feature_result_tg = nfeature_accuracy_checker(stop_words=my_stop_words, ngram_range=(1, 3))

In [ ]:
print("RESULT FOR TRIGRAM WITH STOP WORDS\n")
feature_result_tg2 = nfeature_accuracy_checker(stop_words='english', ngram_range=(1, 3))

In [ ]:
nfeatures_plot_tg = pd.DataFrame(feature_result_tg,columns=['nfeatures','validation_accuracy'])
nfeatures_plot_bg = pd.DataFrame(feature_result_bg,columns=['nfeatures','validation_accuracy'])
nfeatures_plot_ug = pd.DataFrame(feature_result_wocsw,columns=['nfeatures','validation_accuracy'])
nfeatures_plot_tg2 = pd.DataFrame(feature_result_tg2,columns=['nfeatures','validation_accuracy'])
nfeatures_plot_bg2 = pd.DataFrame(feature_result_bg2,columns=['nfeatures','validation_accuracy'])
nfeatures_plot_ug2 = pd.DataFrame(feature_result_wosw,columns=['nfeatures','validation_accuracy'])
plt.figure(figsize=(8,6))
plt.plot(nfeatures_plot_tg.nfeatures, nfeatures_plot_tg.validation_accuracy,label='trigram without custom sw')
plt.plot(nfeatures_plot_bg.nfeatures, nfeatures_plot_bg.validation_accuracy,label='bigram without custom sw')
plt.plot(nfeatures_plot_ug.nfeatures, nfeatures_plot_ug.validation_accuracy, label='unigram without custom sw')
plt.plot(nfeatures_plot_tg2.nfeatures, nfeatures_plot_tg2.validation_accuracy,label='trigram without sw')
plt.plot(nfeatures_plot_bg2.nfeatures, nfeatures_plot_bg2.validation_accuracy,label='bigram without sw')
plt.plot(nfeatures_plot_ug2.nfeatures, nfeatures_plot_ug2.validation_accuracy, label='unigram without sw')
plt.title("N-gram(1~3) test result : Accuracy")
plt.xlabel("Number of features")
plt.ylabel("Validation set accuracy")
plt.legend()

In [ ]:
def train_test_and_evaluate(pipeline, x_train, y_train, x_test, y_test):
    if y_val_len0 / (len(x_test)*1.) > 0.5:
        null_accuracy =  y_val_len1/ (len(x_test)*1.)
    else:
        null_accuracy = 1. - (y_val_len1 / (len(x_test)*1.))
    sentiment_fit = pipeline.fit(x_train, y_train)
    y_pred = sentiment_fit.predict(x_test)
    accuracy = accuracy_score(y_test, y_pred)
    conmat = np.array(confusion_matrix(y_test, y_pred, labels=[0,1]))
    confusion = pd.DataFrame(conmat, index=['non_covid', 'covid'],
                         columns=['predicted_non_covid','predicted_covid'])
    print ("null accuracy: {0:.2f}%".format(null_accuracy*100))
    print ("accuracy score: {0:.2f}%".format(accuracy*100))
    if accuracy > null_accuracy:
        print ("model is {0:.2f}% more accurate than null accuracy".format((accuracy-null_accuracy)*100))
    elif accuracy == null_accuracy:
        print ("model has the same accuracy with the null accuracy")
    else:
        print ("model is {0:.2f}% less accurate than null accuracy".format((null_accuracy-accuracy)*100))
    print ("-"*80)
    print ("Confusion Matrix\n")
    print (confusion)
    print ("-"*80)
    print ("Classification Report\n")
    print (classification_report(y_test, y_pred, target_names=['negative','positive']))

In [ ]:
%%time
tg_cvec = CountVectorizer(max_features=4000,ngram_range=(1, 3))
tg_pipeline = Pipeline([
        ('vectorizer', tg_cvec),
        ('classifier', lr)
    ])
train_test_and_evaluate(tg_pipeline, x_train, y_train, x_validation, y_validation)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tvec = TfidfVectorizer()

In [ ]:
%%time
print ("RESULT FOR UNIGRAM WITHOUT CUSTOM STOP WORDS (Tfidf)\n")
feature_result_ugt = nfeature_accuracy_checker(stop_words = my_stop_words,vectorizer=tvec)

In [ ]:
%%time
print ("RESULT FOR UNIGRAM WITHOUT STOP WORDS (Tfidf)\n")
feature_result_ugt2 = nfeature_accuracy_checker(stop_words = 'english',vectorizer=tvec)

In [ ]:
%%time
print ("RESULT FOR UNIGRAM WITH STOP WORDS (Tfidf)\n")
feature_result_ugt3 = nfeature_accuracy_checker(vectorizer=tvec)

In [ ]:
%%time
print ("RESULT FOR BIGRAM WITHOUT CUSTOM STOP WORDS (Tfidf)\n")
feature_result_bgt = nfeature_accuracy_checker(stop_words = my_stop_words,vectorizer=tvec,ngram_range=(1, 2))

In [ ]:
%%time
print ("RESULT FOR BIGRAM WITHOUT STOP WORDS (Tfidf)\n")
feature_result_bgt2 = nfeature_accuracy_checker(stop_words = 'english',vectorizer=tvec,ngram_range=(1, 2))

In [ ]:
%%time
print ("RESULT FOR BIGRAM WITH STOP WORDS (Tfidf)\n")
feature_result_bgt3 = nfeature_accuracy_checker(vectorizer=tvec,ngram_range=(1, 2))

In [ ]:
%%time
print ("RESULT FOR TRIGRAM WITHOUT CUSTOM  STOP WORDS (Tfidf)\n")
feature_result_tgt = nfeature_accuracy_checker(stop_words = my_stop_words,vectorizer=tvec,ngram_range=(1, 3))

In [ ]:
%%time
print ("RESULT FOR TRIGRAM WITHOUT STOP WORDS (Tfidf)\n")
feature_result_tgt2 = nfeature_accuracy_checker(stop_words = 'english',vectorizer=tvec,ngram_range=(1, 3))

In [ ]:
%%time
print ("RESULT FOR TRIGRAM WITH STOP WORDS (Tfidf)\n")
feature_result_tgt3 = nfeature_accuracy_checker(vectorizer=tvec,ngram_range=(1, 3))

In [ ]:
nfeatures_plot_ugt = pd.DataFrame(feature_result_ugt,columns=['nfeatures','validation_accuracy'])
nfeatures_plot_ugt2 = pd.DataFrame(feature_result_ugt2,columns=['nfeatures','validation_accuracy'])
nfeatures_plot_ugt3 = pd.DataFrame(feature_result_ugt3,columns=['nfeatures','validation_accuracy'])
plt.figure(figsize=(8,6))
plt.plot(nfeatures_plot_ugt.nfeatures, nfeatures_plot_ugt.validation_accuracy, label='without custom stop words')
plt.plot(nfeatures_plot_ugt2.nfeatures, nfeatures_plot_ugt2.validation_accuracy,label='without stop words')
plt.plot(nfeatures_plot_ugt3.nfeatures, nfeatures_plot_ugt3.validation_accuracy,label='with stop words')
plt.title(" tfidf Without stop words VS With stop words (Unigram): Accuracy")
plt.xlabel("Number of features")
plt.ylabel("Validation set accuracy")
plt.legend()

In [ ]:
nfeatures_plot_bgt = pd.DataFrame(feature_result_bgt,columns=['nfeatures','validation_accuracy'])
nfeatures_plot_bgt2 = pd.DataFrame(feature_result_bgt2,columns=['nfeatures','validation_accuracy'])
nfeatures_plot_bgt3 = pd.DataFrame(feature_result_bgt3,columns=['nfeatures','validation_accuracy'])
plt.figure(figsize=(8,6))
plt.plot(nfeatures_plot_bgt.nfeatures, nfeatures_plot_bgt.validation_accuracy, label='without custom stop words')
plt.plot(nfeatures_plot_bgt2.nfeatures, nfeatures_plot_bgt2.validation_accuracy,label='without stop words')
plt.plot(nfeatures_plot_bgt3.nfeatures, nfeatures_plot_bgt3.validation_accuracy,label='with stop words')
plt.title(" tfidf Without stop words VS With stop words (Bigram): Accuracy")
plt.xlabel("Number of features")
plt.ylabel("Validation set accuracy")
plt.legend()

In [ ]:
nfeatures_plot_tgt = pd.DataFrame(feature_result_tgt,columns=['nfeatures','validation_accuracy'])
nfeatures_plot_tgt2 = pd.DataFrame(feature_result_tgt2,columns=['nfeatures','validation_accuracy'])
nfeatures_plot_tgt3 = pd.DataFrame(feature_result_tgt3,columns=['nfeatures','validation_accuracy'])
plt.figure(figsize=(8,6))
plt.plot(nfeatures_plot_tgt.nfeatures, nfeatures_plot_tgt.validation_accuracy, label='without custom stop words')
plt.plot(nfeatures_plot_tgt2.nfeatures, nfeatures_plot_tgt2.validation_accuracy,label='without stop words')
plt.plot(nfeatures_plot_tgt3.nfeatures, nfeatures_plot_tgt3.validation_accuracy,label='with stop words')
plt.title(" tfidf Without stop words VS With stop words (Trigram): Accuracy")
plt.xlabel("Number of features")
plt.ylabel("Validation set accuracy")
plt.legend()

In [ ]:
nfeatures_plot_tgt2 = pd.DataFrame(feature_result_tgt2,columns=['nfeatures','validation_accuracy'])
nfeatures_plot_bgt2 = pd.DataFrame(feature_result_bgt2,columns=['nfeatures','validation_accuracy'])
nfeatures_plot_ugt2 = pd.DataFrame(feature_result_ugt2,columns=['nfeatures','validation_accuracy'])

plt.figure(figsize=(8,6))
plt.plot(nfeatures_plot_tgt2.nfeatures, nfeatures_plot_tgt2.validation_accuracy,label='trigram tfidf vectorizer',color='royalblue')
plt.plot(nfeatures_plot_tg2.nfeatures, nfeatures_plot_tg2.validation_accuracy,label='trigram count vectorizer',linestyle=':', color='royalblue')
plt.plot(nfeatures_plot_bgt2.nfeatures, nfeatures_plot_bgt2.validation_accuracy,label='bigram tfidf vectorizer',color='orangered')
plt.plot(nfeatures_plot_bg2.nfeatures, nfeatures_plot_bg2.validation_accuracy,label='bigram count vectorizer',linestyle=':',color='orangered')
plt.plot(nfeatures_plot_ugt2.nfeatures, nfeatures_plot_ugt2.validation_accuracy, label='unigram tfidf vectorizer',color='gold')
plt.plot(nfeatures_plot_ug_wosw.nfeatures, nfeatures_plot_ug_wosw.validation_accuracy, label='unigram count vectorizer',linestyle=':',color='gold')
plt.title(" Without using stop words N-gram(1~3) test result : Accuracy")
plt.xlabel("Number of features")
plt.ylabel("Validation set accuracy")
plt.legend()

In [ ]:
nfeatures_plot_tgt = pd.DataFrame(feature_result_tgt,columns=['nfeatures','validation_accuracy'])
nfeatures_plot_bgt = pd.DataFrame(feature_result_bgt,columns=['nfeatures','validation_accuracy'])
nfeatures_plot_ugt = pd.DataFrame(feature_result_ugt,columns=['nfeatures','validation_accuracy'])

plt.figure(figsize=(8,6))
plt.plot(nfeatures_plot_tgt.nfeatures, nfeatures_plot_tgt.validation_accuracy,label='trigram tfidf vectorizer',color='royalblue')
plt.plot(nfeatures_plot_tg.nfeatures, nfeatures_plot_tg.validation_accuracy,label='trigram count vectorizer',linestyle=':', color='royalblue')
plt.plot(nfeatures_plot_bgt.nfeatures, nfeatures_plot_bgt.validation_accuracy,label='bigram tfidf vectorizer',color='orangered')
plt.plot(nfeatures_plot_bg.nfeatures, nfeatures_plot_bg.validation_accuracy,label='bigram count vectorizer',linestyle=':',color='orangered')
plt.plot(nfeatures_plot_ugt.nfeatures, nfeatures_plot_ugt.validation_accuracy, label='unigram tfidf vectorizer',color='gold')
plt.plot(nfeatures_plot_ug_wocsw.nfeatures, nfeatures_plot_ug_wocsw.validation_accuracy, label='unigram count vectorizer',linestyle=':',color='gold')
plt.title("Without using custom stop words N-gram(1~3) test result : Accuracy")
plt.xlabel("Number of features")
plt.ylabel("Validation set accuracy")
plt.legend()

In [ ]:
feature_result_bg3 = nfeature_accuracy_checker(ngram_range=(1, 2))
feature_result_tg3 = nfeature_accuracy_checker(ngram_range=(1, 3))

In [ ]:
nfeatures_plot_tgt3 = pd.DataFrame(feature_result_tgt3,columns=['nfeatures','validation_accuracy'])
nfeatures_plot_bgt3 = pd.DataFrame(feature_result_bgt3,columns=['nfeatures','validation_accuracy'])
nfeatures_plot_ugt3= pd.DataFrame(feature_result_ugt3,columns=['nfeatures','validation_accuracy'])

nfeatures_plot_tg3 = pd.DataFrame(feature_result_tg3,columns=['nfeatures','validation_accuracy'])
nfeatures_plot_bg3 = pd.DataFrame(feature_result_bg3,columns=['nfeatures','validation_accuracy'])

plt.figure(figsize=(8,6))
plt.plot(nfeatures_plot_tgt3.nfeatures, nfeatures_plot_tgt3.validation_accuracy,label='trigram tfidf vectorizer',color='royalblue')
plt.plot(nfeatures_plot_tg3.nfeatures, nfeatures_plot_tg3.validation_accuracy,label='trigram count vectorizer',linestyle=':', color='royalblue')
plt.plot(nfeatures_plot_bgt3.nfeatures, nfeatures_plot_bgt3.validation_accuracy,label='bigram tfidf vectorizer',color='orangered')
plt.plot(nfeatures_plot_bg3.nfeatures, nfeatures_plot_bg3.validation_accuracy,label='bigram count vectorizer',linestyle=':',color='orangered')
plt.plot(nfeatures_plot_ugt3.nfeatures, nfeatures_plot_ugt3.validation_accuracy, label='unigram tfidf vectorizer',color='gold')
plt.plot(nfeatures_plot_ug.nfeatures, nfeatures_plot_ug.validation_accuracy, label='unigram count vectorizer',linestyle=':',color='gold')
plt.title("Using stop words N-gram(1~3) test result : Accuracy")
plt.xlabel("Number of features")
plt.ylabel("Validation set accuracy")
plt.legend()

In [ ]:
def accuracy_summary2(pipeline, x_train, y_train, x_test, y_test):
    if y_val_len0 / (len(x_test)*1.) > 0.5:
        null_accuracy =  y_val_len1/ (len(x_test)*1.)
    else:
        null_accuracy = 1. - (y_val_len1 / (len(x_test)*1.))
    #t0 = time.time()
    sentiment_fit = pipeline.fit(x_train, y_train)
    y_pred = sentiment_fit.predict(x_test)
    #train_test_time = time() - t0
    accuracy = accuracy_score(y_test, y_pred)
    print ("null accuracy: {0:.2f}%".format(null_accuracy*100))
    print ("accuracy score: {0:.2f}%".format(accuracy*100))
    if accuracy > null_accuracy:
        print ("model is {0:.2f}% more accurate than null accuracy".format((accuracy-null_accuracy)*100))
    elif accuracy == null_accuracy:
        print ("model has the same accuracy with the null accuracy")
    else:
        print ("model is {0:.2f}% less accurate than null accuracy".format((null_accuracy-accuracy)*100))
    #print ("train and test time: {0:.2f}s".format(train_test_time))
    print ("-"*80)
    return accuracy, sentiment_fit

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import LogisticRegression

clf1 = LogisticRegression()
clf2 = LinearSVC()
clf3 = MultinomialNB()
clf4 = RidgeClassifier()
clf5 = PassiveAggressiveClassifier()

eclf = VotingClassifier(estimators=[('lr', clf1), ('svc', clf2), ('mnb', clf3), ('rcs', clf4), ('pac', clf5)], voting='hard')

for clf, label in zip([clf1, clf2, clf3, clf4, clf5, eclf], ['Logistic Regression', 'Linear SVC', 'Multinomial NB', 'Ridge Classifier', 'Passive Aggresive Classifier', 'Ensemble']):
    checker_pipeline = Pipeline([
            ('vectorizer', TfidfVectorizer(stop_words = 'english',max_features=2000,ngram_range=(1, 3))),
            ('classifier', clf)
        ])
    print ("Validation result for {}".format(label))
    print (clf)
    clf_accuracy, model = accuracy_summary2(checker_pipeline, x_train, y_train, x_validation, y_validation)